# Dogs vs Cats
In this section, we'll start by naively training a small convnet on the 2000 training samples, without any regularization, to set a base-line for what can be achieved. This will get us to a classification accuracy of 71%. At that point, the main issue will be overfitting. Then we'll introduce *data augmentation* , a powerful technique for mitigating overfitting in computer vision. By using data augmentation, we'll improve the network to reach an accuracy of 82%.

In next section, we'll review two more essential techniques for applying deep learning to small datasets: *feature extraction with a pretrained network* (which will get an accuracy of 90% to 96%) and *fine-tuning a pretrained network* (97%). Together, there three strategies will constitute our future toolbox for tackling the problem of performing image classification with small datasets.

In [ ]:
import tensorflow as tf
from functools import partial

__The following code does not run twice__

Making small dataset folder

In [31]:
import os, shutil, pathlib

original_dir = pathlib.Path("/home/ubuntu/dlp_fc/chpt5/data/kaggle_original_data/train/train")
new_base_dir = pathlib.Path("/home/ubuntu/dlp_fc/chpt5/data/cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

This is a balanced __binary-classification__ problem, which means classification accuracy will be an appropriate measure of success.

The depth of the feature maps progressively increases in the network (from 32 to 128), whereas the size of the feature maps decreases (from $148^2$ to $7^2$). This's a pattern you'll see in almost all convnets.

In [32]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Configuring the model for training

In [33]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

### Data preprocessing

Data should be formatted into appropriately preprocessed floating-point tensors before being fed into the network. The steps for getting data into the network are roughly as follows:
1. Read the pic files.
2. Decode the JPEG content to RGB grids of pixels.
3. Convert these into floating-point tensors.
4. Rescale the pixels values (between 0 and 255) to the [0, 1] interval.

In [34]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [35]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


Fitting the model using a batch generator

In [ ]:
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset)

Epoch 1/30
63/63 [==============================] - 8s 109ms/step - loss: 0.7067 - accuracy: 0.5205 - val_loss: 0.6851 - val_accuracy: 0.5010
Epoch 2/30
63/63 [==============================] - 7s 105ms/step - loss: 0.7172 - accuracy: 0.5570 - val_loss: 0.6887 - val_accuracy: 0.5110
Epoch 3/30
63/63 [==============================] - 7s 105ms/step - loss: 0.6623 - accuracy: 0.6125 - val_loss: 0.9464 - val_accuracy: 0.5040
Epoch 4/30
63/63 [==============================] - 7s 104ms/step - loss: 0.6438 - accuracy: 0.6420 - val_loss: 0.6351 - val_accuracy: 0.6200
Epoch 5/30
63/63 [==============================] - 7s 106ms/step - loss: 0.6141 - accuracy: 0.6735 - val_loss: 0.6343 - val_accuracy: 0.6420
Epoch 6/30
63/63 [==============================] - 7s 106ms/step - loss: 0.5795 - accuracy: 0.6965 - val_loss: 0.6177 - val_accuracy: 0.6640
Epoch 7/30
63/63 [==============================] - 7s 104ms/step - loss: 0.5462 - accuracy: 0.7290 - val_loss: 0.6909 - val_accuracy: 0.6670
Epoch 

In [35]:
tf.__version__

'2.5.0'